### Must Run these Cells Before Coding

In [ ]:
import xarray as xr # for reading and loading dataset

# general number libraries
import pandas as pd
import numpy as np

# for mapping
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# calculating distance
import geopandas as gpd
from shapely.geometry import Point, MultiLineString
from scipy.spatial import cKDTree

In [ ]:
# open file
df = xr.open_dataset("AQUA_MODIS.20230101_20230131.L3m.MO.CHL.chlor_a.4km.nc")
df # read the datafile

In [ ]:
# store chlor_a dataset in variable first
chlor_a = df.chlor_a #(can call chlor_a instead of df.chlor_a)

### Just some maps

In [ ]:
## plotting the entire month of janurary
ds.chlor_a.plot()

In [ ]:
## plotting the chloraphyll in just california
# lat = (20, 40)
# lon = (-150, -100)

#copy the plot
CA_chl = ds.chlor_a.copy()

# # plotting it
subset = ds.chlor_a.sel(lat=slice(50, 10), lon=slice(-180, -110)) # slices until the lon and lat coordinates
subset = subset.where(~np.isnan(subset))  # filters out the NAN values
#print(subset)
subset.plot()

#print(ds)

# ds.chlor_a.sel(lat=slice(20, 40), lon=slice(-150, -100)).plot()



### chatgbt code tryout ##

In [ ]:
#customizing plots
ds.chlor_a.plot(cmap='viridis', robust=True)

In [ ]:
# overlaying longitude latitude on map
plt.figure(figsize=(10,6))
ax = plt.axes(projection=ccrs.PlateCarree()) # map projection

ds.chlor_a.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='YlGnBu')
    #ccrs.PlateCarree() – Simple map projection for geospatial data.

ax.coastlines() # Adds coastlines to make it look nice.
plt.show



In [ ]:
# packages
import numpy as mp
import xarray as xr
from shapely.geometry import Point, MultiLineString
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# open the dataset
ds = xr.open_dataset("AQUA_MODIS.20230101_20230131.L3m.MO.CHL.chlor_a.4km.nc")
chls = ds["chlor_a"].values
lats = ds["lat"].values
lons = ds["lon"].values

# getting ocean points
''' creates a point in ocean with lat and lon in the ocean (by checking the chl value)'''
ocean_points = [ (lat, lon)
    for lat, lon, chl in zip(lats.flatten(), lons.flatten(), chls.flatten()) # flatten <- creates a 1D array from a 2D data
    if not np.isnan(chl) # if the data isn't a NAN (this means that the data is ocean)
]

# load coastline data using Cartopy's Natural Earth dataset
shp_path = shpreader.natural_earth(category = "physical", name = "coastline", resolution = "11om")



In [ ]:
import numpy as np
import xarray as xr
from shapely.geometry import Point, MultiLineString
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

def load_chlorophyll_data(file_path):
    """Load chlorophyll dataset and extract valid ocean points."""
    data = xr.open_dataset("AQUA_MODIS.20230101_20230131.L3m.MO.CHL.chlor_a.4km.nc")
    chlorophyll = data["chlor_a"].values
    lats = data["lat"].values
    lons = data["lon"].values

    # Extract ocean points
    ocean_points = [
        (lat, lon)
        for lat, lon, chl in zip(lats.flatten(), lons.flatten(), chlorophyll.flatten()) 
        if not np.isnan(chl)
    ]

    return ocean_points, lats, lons, chlorophyll

def load_coastline():
    """Load coastline data using Cartopy's Natural Earth dataset."""
    shp_path = shpreader.natural_earth(category="physical", name="coastline", resolution="110m")
    coastline_shapes = list(shpreader.Reader(shp_path).geometries())
    return MultiLineString(coastline_shapes)

def calculate_distances(ocean_points, coastline):
    """Calculate the distance from each ocean point to the nearest coastline."""
    distances = []
    for lat, lon in ocean_points:
        point = Point(lon, lat)
        # Approximate distance using geodetic projection (in degrees to meters)
        distances.append(coastline.project(point))
    return distances

def map_distances_to_grid(ocean_points, distances, lats, lons, chlorophyll):
    """Map distances back into a grid matching the chlorophyll data."""
    distance_grid = np.full(chlorophyll.shape, np.nan)
    for i, (lat, lon) in enumerate(ocean_points):
        row_idx = np.argmin(np.abs(lats - lat))
        col_idx = np.argmin(np.abs(lons - lon))
        distance_grid[row_idx, col_idx] = distances[i]
    return distance_grid

def plot_distance_map(distance_grid, lons, lats):
    """Plot the distance grid on a map."""
    plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_title("Distance to Coastline")
    ax.coastlines(resolution="110m")
    distance_plot = ax.contourf(lons, lats, distance_grid, cmap="viridis", transform=ccrs.PlateCarree())
    plt.colorbar(distance_plot, label="Distance (meters)")
    plt.show()

def main():
    """Main function to calculate and visualize distances from ocean points to coastline."""
    # File path to chlorophyll dataset
    file_path = "AQUA_MODIS.20230101_20230131.L3m.MO.CHL.chlor_a.4km.nc"

    # Step 1: Load data
    ocean_points, lats, lons, chlorophyll = load_chlorophyll_data(file_path)

    # Step 2: Load coastline
    coastline = load_coastline()

    # Step 3: Calculate distances
    distances = calculate_distances(ocean_points, coastline)

    # Step 4: Map distances to grid
    distance_grid = map_distances_to_grid(ocean_points, distances, lats, lons, chlorophyll)

    # Step 5: Visualize
    plot_distance_map(distance_grid, lons, lats)

if __name__ == "__main__":
    main()




In [ ]:
from shapely.geometry import Point, MultiLineString, LineString
line1 = LineString([(0, 0), (1, 1), (2, 2)])
line2 = LineString([(2, 2), (3, 3), (4, 4)])
line3 = LineString([(0, 1), (1, 2), (2, 3)])

# Combine them into a MultiLineString
multi_line = MultiLineString([line1, line2, line3])

# Print the MultiLineString
print("MultiLineString:", multi_line)

# Get the number of LineStrings
print("Number of LineStrings:", len(multi_line.geoms))

# Access each LineString
for i, line in enumerate(multi_line.geoms):
    print(f"Line {i+1}: {line}")

# Define a point
point = Point(3, 2)

# Calculate the distance
distance = multi_line.distance(point)
print("Distance from point to MultiLineString:", distance)

# Plot the MultiLineString
fig, ax = plt.subplots()
for line in multi_line.geoms:
    x, y = line.xy
    ax.plot(x, y, label="LineString", color="blue")

# Plot the point
ax.plot(3, 2, 'ro', label="Point")  # Red point at (3, 2)

# Customize the plot
ax.set_title("MultiLineString Visualization")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import math

points = [(1, 2), (3, 5), (2, 8), (6, 3)]

# Calculate distances between all pairs of points
distances = []
for i in range(len(points)):
    for j in range(i + 1, len(points)):
        distance = math.dist(points[i], points[j])
        distances.append((i, j, distance))

# Plot the points
x_coords, y_coords = zip(*points)
plt.scatter(x_coords, y_coords, color='blue', label='Points')

# Annotate distances on the plot
for i, j, distance in distances:
    x1, y1 = points[i]
    x2, y2 = points[j]
    plt.plot([x1, x2], [y1, y2], 'k--', alpha=0.5)
    plt.text((x1 + x2) / 2, (y1 + y2) / 2, f'{distance:.2f}', fontsize=8, ha='center')

plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Distances Between Points')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiLineString
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
from shapely.ops import unary_union
from cartopy.io import shapereader as shpreader

# Load chlorophyll data (example)
ds = xr.open_dataset("AQUA_MODIS.20230101_20230131.L3m.MO.CHL.chlor_a.4km.nc")
chlor_a = ds.chlor_a

# Subset the data to a specific region (latitudes and longitudes)
subset = chlor_a.sel(lat=slice(50, 10), lon=slice(-180, -110))

# Load coastline data using Cartopy's Natural Earth dataset
shp_path = shpreader.natural_earth(resolution="110m", category="physical", name="coastline")
coastline_shapes = list(shpreader.Reader(shp_path).geometries())
coastline = unary_union(MultiLineString(coastline_shapes))  # Combine into a single MultiLineString

# Extract chlor_a coordinates (lat, lon) and flatten them
lats, lons = subset['lat'].values, subset['lon'].values
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Create a list of valid (non-NaN) points
valid_points = [
    Point(lon, lat)
    for lon, lat, chl in zip(lon_grid.flatten(), lat_grid.flatten(), subset.values.flatten())
    if not np.isnan(chl)
]

# Extract all coordinates from the MultiLineString
coastline_coords = np.array([coord for line in coastline for coord in line.coords])

# Build a KDTree for efficient distance calculations
tree = cKDTree(coastline_coords)

# Calculate distances for all valid points
distances = np.array([tree.query((point.x, point.y))[0] for point in valid_points])

# Create a distance grid (NaN for invalid points)
distance_grid = np.full(lon_grid.shape, np.nan)
for (i, j), distance in zip(
    [(np.where(lat_grid == point.y)[0][0], np.where(lon_grid == point.x)[0][0]) for point in valid_points],
    distances,
):
    distance_grid[i, j] = distance

# Add distances to the dataset as a new variable
ds['distance_to_coastline'] = (('lat', 'lon'), distance_grid)

# Plot the distances
plt.figure(figsize=(10, 6))
ds['distance_to_coastline'].plot(cmap='viridis')
plt.title("Distance to Coastline (degrees)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

## Plotting Distance

** TO DO **
- x axis representing distance
- y axis representing concentration
- calculate the avg of how far a concentration of Chlor is

**California region coordinates**
- lat_range = (20, 40)     # From Baja to Northern California
- lon_range = (-130, -110)  # Coastal Pacific region

In [ ]:
# chatgpt
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiLineString
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
from cartopy.io import shapereader as shpreader
from shapely.ops import unary_union

# 1. Load chlorophyll data
ds = xr.open_dataset("AQUA_MODIS.20230101_20230131.L3m.MO.CHL.chlor_a.4km.nc")
chlor_a = ds.chlor_a

# 2. Subset data for a specific region (lat: 50 to 10, lon: -180 to -110)
subset = chlor_a.sel(lat=slice(50, 10), lon=slice(-180, -110))

# 3. Load coastline data
shp_path = shpreader.natural_earth(resolution="110m", category="physical", name="coastline")
coastline_shapes = list(shpreader.Reader(shp_path).geometries())
coastline = unary_union(MultiLineString(coastline_shapes))  # Combine into a single MultiLineString

# 4. Extract lat/lon and filter out NaN chlorophyll values
lats, lons = subset['lat'].values, subset['lon'].values
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Create valid points (filter out NaN chlorophyll values)
valid_points = [
    Point(lon, lat)
    for lon, lat, chl in zip(lon_grid.flatten(), lat_grid.flatten(), subset.values.flatten())
    if not np.isnan(chl)
]
valid_chlorophyll = [
    chl
    for chl in subset.values.flatten()
    if not np.isnan(chl)
]


In [ ]:
## in degrees
# 5. Calculate distances from points to the coastline
def calculate_distances(points, coastline):
    """Calculate distances from points to the coastline using cKDTree."""
    # Extract all coordinates from the MultiLineString
    coastline_coords = np.array(
        [coord for line in coastline.geoms for coord in line.coords]
    )
    
    # Build a KDTree from the coastline coordinates
    tree = cKDTree(coastline_coords)
    
    # Calculate the shortest distance from each point to the coastline
    distances = np.array([tree.query((point.x, point.y))[0] for point in points])
    return distances

distances = calculate_distances(valid_points, coastline)


In [ ]:

# 6. Create the X-Y plot (distance vs. chlorophyll concentration)
plt.figure(figsize=(10, 6))
plt.scatter(distances, valid_chlorophyll, s=5, alpha=0.7, c=distances, cmap='viridis')
plt.colorbar(label='Distance to Coastline (degrees)')
plt.xlabel("Distance to Coastline (degrees)")
plt.ylabel("Chlorophyll Concentration")
plt.title("Distance to Coastline vs Chlorophyll Concentration")
plt.show()

#### log scale distance to coastline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create some example data
x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y = 2**x  # This will create exponential growth: 2, 4, 8, 16, 32, ...

# Let's create two subplots to compare regular scale vs log scale
plt.figure(figsize=(12, 5))

# Regular scale plot
plt.subplot(121)  # 1 row, 2 columns, first plot
plt.plot(x, y, 'bo-')
plt.title('Regular Scale')
plt.grid(True)

# Log scale plot
plt.subplot(122)  # 1 row, 2 columns, second plot
plt.loglog(x, y, 'ro-')
plt.title('Log-Log Scale')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
plt.loglog(distances, valid_chlorophyll, 'go', markersize=1)


#### using unyt to convert degrees to meters

In [ ]:
from unyt import unyt_array
from unyt.unit_symbols import degree, m

distance_with_units = unyt_array(distances, degree) # tells unyt distance are in degrees


In [ ]:
#chatgbt
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiLineString
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
from cartopy.io import shapereader as shpreader
from shapely.ops import unary_union
from unyt import deg, m, rad
import math
# 5. Calculate distances from points to the coastline with unyt
def calculate_distances_with_units(points, coastline):
    """Calculate distances from points to the coastline using cKDTree and convert to meters."""
    # Extract all coordinates from the MultiLineString
    coastline_coords = np.array(
        [coord for line in coastline.geoms for coord in line.coords]
    )
    
    # Build a KDTree from the coastline coordinates
    tree = cKDTree(coastline_coords)
    
    # Initialize a list to store distances
    distances_in_meters = []
    
    for point in points:
        # Find the nearest coastline point (distance in degrees)
        dist_deg = tree.query((point.x, point.y))[0] * deg  # Attach "degree" unit to distance
        
        # Convert latitude (point.y) to radians for longitude scaling
        latitude_rad = point.y * (np.pi / 180)  # Convert latitude from degrees to radians
        
        # Convert distance from degrees to meters
        # Latitude distance: 1 degree latitude ≈ 111,000 meters
        lat_distance = dist_deg.value * 111_000 * m
        
        # Longitude distance: scaled by cos(latitude)
        lon_distance = dist_deg.value * 111_000 * np.cos(latitude_rad) * m
        
        # Use Pythagorean theorem for total distance in meters
        total_distance = np.sqrt(lat_distance**2 + lon_distance**2)
        distances_in_meters.append(total_distance)
    
    return np.array(distances_in_meters)

distances_in_meters = calculate_distances_with_units(valid_points, coastline)

In [ ]:
# 6. Create the X-Y plot (distance vs. chlorophyll concentration)
plt.figure(figsize=(10, 6))
plt.scatter(distances, valid_chlorophyll, s=5, alpha=0.7, c=distances, cmap='viridis')
plt.colorbar(label='Distance to Coastline (meters)')
plt.xlabel("Distance to Coastline (meters)")
plt.ylabel("Chlorophyll Concentration")
plt.title("Distance to Coastline vs Chlorophyll Concentration")
plt.show()

In [ ]:
from matplotlib.patches import Rectangle

import matplotlib.pyplot as plt

# Create a new figure and axis
fig, ax = plt.subplots()

# Create a square (Rectangle with equal width and height)
square = Rectangle((0.1, 0.1), 0.5, 0.5, edgecolor='r', facecolor='none')

# Add the square to the plot
ax.add_patch(square)

# Set the limits of the plot
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Set aspect of the plot to be equal
ax.set_aspect('equal')

# Show the plot
plt.show()

# testing changes